In [1]:
from flask import Flask, request, render_template
import cv2
import numpy as np


ModuleNotFoundError: No module named 'cv2'

In [ ]:
app = Flask(__name__)


In [ ]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]


In [ ]:
@app.route('/')
def home():
    return render_template('index.html')


In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    # Load the uploaded image
    image = request.files['image'].read()
    nparr = np.frombuffer(image, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # Detect objects using YOLO
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Process the detected objects
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                                # Object detected, get object details
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Calculate coordinates of bounding box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Add object details to respective lists
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    # Non-maximum suppression to remove overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Prepare response data for rendering on web page
    detected_objects = []
    for i in indices:
        i = i[0]
        label = str(classes[class_ids[i]])
        confidence = confidences[i]
        box = boxes[i]
        x, y, w, h = box
        detected_objects.append({"label": label, "confidence": confidence, "x": x, "y": y, "w": w, "h": h})

    # Render the response on web page
    return render_template('result.html', detected_objects=detected_objects)

